In [ ]:
!pip3 install raster2xyz

In [10]:
import pandas as pd
import geopandas as gpd
import cdsapi
import xarray as xr
from urllib.request import urlopen
import datetime
import random
import cdsapi
import rasterio
from rasterio import features
from raster2xyz.raster2xyz import Raster2xyz
from shapely.geometry import Point
import pyproj

In [2]:
# Specify the paths to the GeoTIFF files (original coordinate system EPSG:3042)
elevation_file = "data/municipios/cartografia_tridimensional/03102_cartografia_tridimensional/MDT_municipio/MDT_03102.tif"
slope_file = "data/municipios/cartografia_tridimensional/03102_cartografia_tridimensional/pendiente_municipio/pendiente_03102.tif"
orientation_file = "data/municipios/cartografia_tridimensional/03102_cartografia_tridimensional/orientacion_municipio/orientacion_03102.tif"

In [3]:
# Altitud

elevation_csv = "data/municipios/datos_generados/03102/altitud_03102.csv"
rtxyz = Raster2xyz()
rtxyz.translate(elevation_file, elevation_csv)

# Pendiente

slope_csv = "data/municipios/datos_generados/03102/pendiente_03102.csv"
rtxyz = Raster2xyz()
rtxyz.translate(slope_file, slope_csv)

# Orientacion

orientation_csv = "data/municipios/datos_generados/03102/orientacion_03102.csv"
rtxyz = Raster2xyz()
rtxyz.translate(orientation_file, orientation_csv)

[2023-05-22 03:38:15 - INFO] - Getting geotransform and data...
[2023-05-22 03:38:15 - INFO] - Getting XYZ data...
[2023-05-22 03:38:15 - INFO] - Getting geotransformed coordinates...
[2023-05-22 03:38:15 - INFO] - Building XYZ data...
[2023-05-22 03:38:23 - INFO] - New XYZ (csv file) created...
[2023-05-22 03:38:23 - INFO] - Getting geotransform and data...
[2023-05-22 03:38:23 - INFO] - Getting XYZ data...
[2023-05-22 03:38:23 - INFO] - Getting geotransformed coordinates...
[2023-05-22 03:38:23 - INFO] - Building XYZ data...
[2023-05-22 03:38:31 - INFO] - New XYZ (csv file) created...
[2023-05-22 03:38:31 - INFO] - Getting geotransform and data...
[2023-05-22 03:38:31 - INFO] - Getting XYZ data...
[2023-05-22 03:38:31 - INFO] - Getting geotransformed coordinates...
[2023-05-22 03:38:31 - INFO] - Building XYZ data...
[2023-05-22 03:38:39 - INFO] - New XYZ (csv file) created...


In [4]:
elevation_03102_df = pd.read_csv(elevation_csv)
slope_03102_df = pd.read_csv(slope_csv)
orientation_03102_df = pd.read_csv(orientation_csv)

In [5]:
elevation_03102_df['altitud'] = elevation_03102_df['z']
slope_03102_df['pendiente'] = slope_03102_df['z']
orientation_03102_df['orientacion'] = orientation_03102_df['z']

In [6]:
elevation_03102_df.drop('z', axis=1, inplace=True)
slope_03102_df.drop('z', axis=1, inplace=True)
orientation_03102_df.drop('z', axis=1, inplace=True)

In [7]:
raster_03102_df = elevation_03102_df
raster_03102_df['pendiente'] = slope_03102_df['pendiente']
raster_03102_df['orientacion'] = orientation_03102_df['orientacion']

In [8]:
raster_03102_df

,x,y,altitud,pendiente,orientacion
0,746798.870135,4.308172e+06,425.26700,27.389078,115.62827
1,746803.870135,4.308172e+06,423.99326,30.299067,114.12234
2,746808.870135,4.308172e+06,422.49823,33.705704,111.60547
3,746813.870135,4.308172e+06,420.87625,35.246620,109.49768
4,746818.870135,4.308172e+06,419.19073,37.483860,109.14037
...,...,...,...,...,...
3902323,756733.870135,4.298382e+06,90.00375,10.575702,72.93383
3902324,756738.870135,4.298382e+06,89.24925,14.328173,80.29402
3902325,756743.870135,4.298382e+06,88.55600,5.111823,83.11841
3902326,756748.870135,4.298382e+06,88.75925,8.438879,264.53876


In [11]:
# Define the source and target coordinate systems
source_crs = pyproj.CRS('EPSG:3042')
target_crs = pyproj.CRS('EPSG:4326')

# Create a Proj transformer to convert coordinates
transformer = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)

# Reproject the coordinates to EPSG:4326
reprojected_geometry = []
for x, y in zip(raster_03102_df['x'], raster_03102_df['y']):
    lon, lat = transformer.transform(x, y)
    reprojected_geometry.append(Point(lon, lat))

# Create a GeoDataFrame with the reprojected geometry column
reprojected_gdf = gpd.GeoDataFrame(raster_03102_df, geometry=reprojected_geometry, crs=target_crs)

In [12]:
reprojected_gdf

,x,y,altitud,pendiente,orientacion,geometry
0,746798.870135,4.308172e+06,425.26700,27.389078,115.62827,POINT (-0.15460 38.88778)
1,746803.870135,4.308172e+06,423.99326,30.299067,114.12234,POINT (-0.15454 38.88778)
2,746808.870135,4.308172e+06,422.49823,33.705704,111.60547,POINT (-0.15448 38.88777)
3,746813.870135,4.308172e+06,420.87625,35.246620,109.49768,POINT (-0.15442 38.88777)
4,746818.870135,4.308172e+06,419.19073,37.483860,109.14037,POINT (-0.15437 38.88777)
...,...,...,...,...,...,...
3902323,756733.870135,4.298382e+06,90.00375,10.575702,72.93383,POINT (-0.04384 38.79682)
3902324,756738.870135,4.298382e+06,89.24925,14.328173,80.29402,POINT (-0.04379 38.79682)
3902325,756743.870135,4.298382e+06,88.55600,5.111823,83.11841,POINT (-0.04373 38.79682)
3902326,756748.870135,4.298382e+06,88.75925,8.438879,264.53876,POINT (-0.04367 38.79682)


In [13]:
reprojected_gdf_cleaned = reprojected_gdf.loc[reprojected_gdf['altitud'] > 0]

In [19]:
sample_df = reprojected_gdf_cleaned.sample(1000)

In [20]:
sample_df.to_file('data/municipios/datos_generados/03102_crs/geo_03102_df.geojson', driver='GeoJSON')